In [1864]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database connection

In [1865]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [1866]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [1867]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

In [1868]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas sin saltos
pd.set_option('display.width', 1000)    

## Extraction

In [1869]:
cliente = pd.read_sql_table('dim_cliente', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
hora = pd.read_sql_table('hecho_accumulating_snapshot', etl_conn)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

In [1870]:
servicio.drop(columns=["descripcion","nombre_solicitante","fecha_deseada","hora_deseada","nombre_recibe",
                       "telefono_recibe", "descripcion_pago","ida_y_regreso","activo","novedades","destino_id",
                       "tipo_pago_id",	"tipo_servicio_id",	"tipo_vehiculo_id",	"usuario_id",	"prioridad",
                           "ciudad_destino_id",	"ciudad_origen_id",	"hora_visto_por_mensajero",	"visto_por_mensajero",
                               	"descripcion_multiples_origenes",	"mensajero2_id",	"mensajero3_id",
                                    "multiples_origenes",	"asignar_mensajero","es_prueba",	"descripcion_cancelado",
                                    "fecha_solicitud","origen_id"],inplace=True)
servicio.head(5)

,id,hora_solicitud,cliente_id,mensajero_id
0,34,09:46:03,5,NaN
1,35,11:18:14,5,7.0
2,36,19:21:01,5,NaN
3,41,09:46:09,5,NaN
4,42,09:46:10,5,NaN


In [1871]:
hora.head(5)

,key_servicio_fase,id,key_mensajero,key_fecha_inicio,key_hora_inicio,key_fecha_mensajero_asignado,key_hora_mensajero_asignado,key_fecha_recogido_mensajero,key_hora_recogido_mensajero,key_fecha_entregado_destino,key_hora_entregado_destino,key_fecha_terminado,key_hora_terminado,dias_mensajero_asignado,horas_mensajero_asignado,dias_recogido_mensajero,horas_recogido_mensajero,dias_entregado_destino,horas_entregado_destino,dias_terminado,horas_terminado,promedio_entrega_general_dias,promedio_entrega_general_horas
0,0,7,0.0,253.0,16.0,277.0,17.0,295.0,12.0,295.0,17.0,295.0,12.0,24.0,1.48,17.0,18.18,0.0,5.08,1.0,19.13,10.5,10.97
1,1,8,13.0,253.0,16.0,345.0,20.0,401.0,15.0,456.0,16.0,NaN,NaN,92.0,3.73,55.0,19.32,55.0,0.57,0.0,0.00,50.5,5.90
2,2,9,13.0,253.0,16.0,353.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0,3.03,0.0,0.00,0.0,0.00,0.0,0.00,25.0,0.76
3,3,10,13.0,253.0,16.0,353.0,19.0,405.0,0.0,426.0,9.0,NaN,NaN,100.0,2.95,51.0,4.80,21.0,9.60,0.0,0.00,43.0,4.34
4,4,11,13.0,253.0,16.0,334.0,13.0,387.0,10.0,NaN,NaN,NaN,NaN,80.0,20.60,52.0,21.25,0.0,0.00,0.0,0.00,33.0,10.46


## Merge Horas

In [1872]:

hecho_servicio_hora = servicio.merge(
    hora[["id", "horas_mensajero_asignado", "key_hora_mensajero_asignado"]],
    on="id", 
    how="left"
)

print(hecho_servicio_hora.head(5))


   id hora_solicitud  cliente_id  mensajero_id  horas_mensajero_asignado  key_hora_mensajero_asignado
0  34       09:46:03           5           NaN                       0.0                          NaN
1  35       11:18:14           5           7.0                       3.4                         14.0
2  36       19:21:01           5           NaN                       0.0                          NaN
3  41       09:46:09           5           NaN                       0.0                          NaN
4  42       09:46:10           5           NaN                       0.0                          NaN


In [1873]:
hecho_servicio_hora = pd.merge(hecho_servicio_hora, cliente[["cliente_id", "key_cliente"]], left_on="cliente_id", right_on="cliente_id", how="left")
hecho_servicio_hora.head(5)

,id,hora_solicitud,cliente_id,mensajero_id,horas_mensajero_asignado,key_hora_mensajero_asignado,key_cliente
0,34,09:46:03,5,NaN,0.0,NaN,7
1,35,11:18:14,5,7.0,3.4,14.0,7
2,36,19:21:01,5,NaN,0.0,NaN,7
3,41,09:46:09,5,NaN,0.0,NaN,7
4,42,09:46:10,5,NaN,0.0,NaN,7


In [1874]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
hecho_servicio_hora = hecho_servicio_hora.merge(mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
hecho_servicio_hora.head(5)

,id,hora_solicitud,cliente_id,mensajero_id,horas_mensajero_asignado,key_hora_mensajero_asignado,key_cliente,key_mensajero
0,34,09:46:03,5,NaN,0.0,NaN,7,NaN
1,35,11:18:14,5,7.0,3.4,14.0,7,13.0
2,36,19:21:01,5,NaN,0.0,NaN,7,NaN
3,41,09:46:09,5,NaN,0.0,NaN,7,NaN
4,42,09:46:10,5,NaN,0.0,NaN,7,NaN


In [1875]:
print(len(hecho_servicio_hora))

28430


## Solicitudes_por_hora

In [1876]:
# Contar los registros por cada 'key_hora_mensajero_asignado'
conteo_por_hora = hecho_servicio_hora.groupby("key_hora_mensajero_asignado").size().reset_index(name="asignacion_por_hora")

# Hacer el merge con el DataFrame original para agregar el conteo como un nuevo atributo
hecho_servicio_hora = pd.merge(hecho_servicio_hora, conteo_por_hora, on="key_hora_mensajero_asignado", how="left")


print(conteo_por_hora.head())


   key_hora_mensajero_asignado  asignacion_por_hora
0                          0.0                  141
1                          1.0                  164
2                          2.0                  152
3                          3.0                  117
4                          4.0                  108


In [1877]:
print(len(hecho_servicio_hora))

28430


## Solicitud_por_hora_cliente

In [1878]:

# Convertir 'hora_solicitud' a un formato decimal
hecho_servicio_hora['hora_solicitud'] = hecho_servicio_hora['hora_solicitud'].apply(lambda x: x.hour + x.minute / 60)

# Guardar solo la parte entera de la hora (sin los minutos)
hecho_servicio_hora['hora_solicitud'] = hecho_servicio_hora['hora_solicitud'].astype(int)

# Contar la cantidad de registros de 'key_cliente' por cada 'hora_solicitud'
solicitud_por_hora_cliente = hecho_servicio_hora.groupby("hora_solicitud")['key_cliente'].count().reset_index(name="solicitud_por_hora_cliente")


hecho_servicio_hora = hecho_servicio_hora.merge(solicitud_por_hora_cliente, on="hora_solicitud", how="left")



solicitud_por_hora_cliente.head(24)


,hora_solicitud,solicitud_por_hora_cliente
0,0,146
1,1,164
2,2,156
3,3,107
4,4,103
5,5,138
6,6,901
7,7,1066
8,8,2636
9,9,3391


In [1879]:
# Contar los valores NaN en la columna "key_hora_mensajero_asignado"
nan_count = hecho_servicio_hora["key_cliente"].isna().sum()

print(f"Cantidad de valores NaN en 'key_hora_mensajero_asignado': {nan_count}")


Cantidad de valores NaN en 'key_hora_mensajero_asignado': 0


In [1880]:
# Agrupar por 'key_hora_mensajero_asignado' y obtener el primer registro por grupo
hora_agrupada = hecho_servicio_hora.groupby("key_hora_mensajero_asignado").agg({
    'asignacion_por_hora': 'first',  # Usamos 'first' para obtener el primer registro de cada grupo
}).reset_index()

# Agrupar por 'key_hora_mensajero_asignado' y obtener el primer registro por grupo
hora_agrupada_solicitud = hecho_servicio_hora.groupby("hora_solicitud").agg({
   
    'solicitud_por_hora_cliente': 'first',  # Obtener el primer valor para cada grupo

}).reset_index()

# Unir ambos DataFrames en un solo DataFrame final
hora_agrupada_final = hora_agrupada.merge(hora_agrupada_solicitud,left_on="key_hora_mensajero_asignado", right_on="hora_solicitud", how="left")


hora_agrupada_final.head(24)


,key_hora_mensajero_asignado,asignacion_por_hora,hora_solicitud,solicitud_por_hora_cliente
0,0.0,141.0,0,146
1,1.0,164.0,1,164
2,2.0,152.0,2,156
3,3.0,117.0,3,107
4,4.0,108.0,4,103
5,5.0,88.0,5,138
6,6.0,367.0,6,901
7,7.0,1321.0,7,1066
8,8.0,3155.0,8,2636
9,9.0,3228.0,9,3391


In [1881]:
hora_agrupada_final.drop(columns=["hora_solicitud"],inplace=True)

## Load

In [1882]:
hora_agrupada_final.to_sql("hecho_servicio_hora", etl_conn, if_exists="replace", index_label="key_servicio_hora") 

24